# Automate Getting Recordings
Or see the [Manual Documentation](https://docs.google.com/document/d/1ZTHdnEKNGF_DOZWCwQdO2sYWhaDCpZbBXDNp01UQe5Y/edit)



In [ ]:

# TODO: Video name should be <unique_name>_<datetime>
# TODO: Upload to Google Drive:
# https://stackoverflow.com/questions/48993906/google-drive-python-api-uploading-large-files

from datetime import date
from datetime import datetime
from datetime import timedelta
import os
from twilio.rest import Client
import json
from urllib.request import urlopen
import time
import pandas as pd

# Compose and Download

In [ ]:

# Your Account SID from twilio.com/console
account_sid = "AC8e7bb5a5223120747ab835398c6e6634"
ua_sid = 'AC61a955ebbab836551663c4521f1f5b41'

# Your Auth Token from twilio.com/console
BEN_JUNTO_DIR = '/Users/benjaminturtel/dev/junto'
TWILIO_CRED_FILE = '%s/creds/twilio_auth_token' % BEN_JUNTO_DIR
auth_token  = open(TWILIO_CRED_FILE, "r").read().strip()

UA_TWILIO_CRED_FILE = '%s/creds/ua_twilio_auth_token' % BEN_JUNTO_DIR
ua_auth_token  = open(UA_TWILIO_CRED_FILE, "r").read().strip()


DOWNLOAD_FOLDER = '/Users/benjaminturtel/Downloads/TwilioCompositions'

def compose_from_room(room_sid):
    # room_sid is typically 'RMXXXX',
    composition = client.video.compositions.create(
        room_sid = room_sid,
        format='mp4',
        audio_sources = '*',
        video_layout ={
            "grid":{
                "video_sources":["*"]
                }})
    print('Created composition with SID=%s' % (composition.sid))


def download_composition(composition_sid, download_location):
    uri = 'https://video.twilio.com/v1/Compositions/{}/Media?Ttl=3600'.format(composition_sid)
    response = client.request('GET', uri)
    media_location = json.loads(response.text).get('redirect_to')

    # For example, get the data and download the media to a local file
    if not os.path.isdir(os.path.dirname(download_location)):
        os.makedirs(os.path.dirname(download_location))
    with open(download_location, 'wb+') as f:
        f.write(urlopen(media_location).read())

In [ ]:

def compose_from_room(room_sid):
    # room_sid is typically 'RMXXXX',
    composition = client.video.compositions.create(
        room_sid = room_sid,
        format='mp4',
        audio_sources = '*',
        video_layout ={
            "grid":{
                "video_sources":["*"]
                }})
    print('Created composition with SID=%s' % (composition.sid))


def download_composition(composition_sid, download_location):
    uri = 'https://video.twilio.com/v1/Compositions/{}/Media?Ttl=3600'.format(composition_sid)
    response = client.request('GET', uri)
    media_location = json.loads(response.text).get('redirect_to')

    # For example, get the data and download the media to a local file
    if not os.path.isdir(os.path.dirname(download_location)):
        os.makedirs(os.path.dirname(download_location))
    with open(download_location, 'wb+') as f:
        f.write(urlopen(media_location).read())

# UA Recordings
* Iterate over UA rooms: Subaccount
* Function compose from rooms
* And a download link


In [ ]:
client = Client(ua_sid, ua_auth_token)

In [ ]:
rooms = client.video.rooms.list(status='completed', limit=1000)# , date_created_after=date(2021, 1, 22), date_created_before=date(2021, 1, 24))
compose_rooms = [r for r in rooms if len(r.participants.list()) >= 2 and r.recordings.list()]
len(compose_rooms)


In [ ]:
print("%d rooms, %d to compose" % (len(rooms), len(compose_rooms)))
for room in compose_rooms:  # Compose all videos
    compose_from_room(room.sid)

In [ ]:
finished = False
while not finished:  # Wait for all to finish
    finished = True
    for room in compose_rooms:
        room_compositions = client.video.compositions.list(room_sid=room.sid)
        if all([c.status == 'completed' for c in room_compositions]):
            print("%s completed" % room.sid)
        else:
            print("%s not completed" % room.sid)
            finished = False
            time.sleep(60)  # Wait 1 minute
            break

In [ ]:

for room in compose_rooms:  # Download all videos
    unique_name = room.unique_name
    comp_list = client.video.compositions.list(room_sid=room.sid)
    for i, comp in enumerate(comp_list):
        filename = unique_name
        if len(comp_list) > 1:
            filename += '_%d' % i
        download_composition(comp.sid, '%s/%s.%s' % (DOWNLOAD_FOLDER, filename, comp.format))

# Brief Report

In [ ]:
# Brief report
rooms = client.video.rooms.list(status='completed', limit=100, date_created_after=date(2021, 1, 18))

# Filter to United America - start with 'i-UC'
ua_rooms = [r for r in rooms if r.unique_name.startswith('i-UC')]

In [ ]:

data = [[r.unique_name, r.date_created, 
    len(set([r.sid for r in r.participants.list()])),
    len(set([r.identity for r in r.participants.list()])),
    ] for r in ua_rooms]
df = pd.DataFrame(data, columns = ['room', 'date', 'p-sids', 'identities']).sort_values('identities', ascending=False).drop_duplicates(['room']).sort_values('date')

In [ ]:
print('\n'.join(df.apply(
    lambda x: 'Room {room}, {date}, {p-sids} sids, {identities} identities.'.format(**x), 1
)))

# AUTH

In [ ]:
from datetime import datetime
from datetime import timedelta
from twilio.rest import Client
from twilio.jwt.access_token import AccessToken
from twilio.jwt.access_token.grants import VideoGrant

ACCOUNT_SID = "TWILIO_ACCOUNT_SID_PLACEHOLDER"
API_KEY = 'TWILIO_API_KEY_PLACEHOLDER'
API_SECRET = 'TWILIO_API_SECRET_PLACEHOLDER'

ttl = 31
client = Client(API_KEY, API_SECRET, ACCOUNT_SID)
# Delete everything older than 31 days
before_date = str((datetime.now() - timedelta(days=ttl)
                    ).replace(microsecond=0).isoformat()) + 'Z'
recordings = client.video.recordings.list(date_created_before=before_date)
for r in recordings:
    print("Delete Recording {}, created {}".format(r.sid, r.date_created))

compositions = client.video.compositions.list(date_created_before=before_date)
for c in compositions:
    print("Delete Composition {}, room {}, created {}".format(c.sid, c.room_sid, c.date_created))